<b>Выполните следующие задания:</b>
- решите задачи регрессии и классификации на ваших данных используя полносвязные нейронные сети; соберите их используя API Keras фреймворка TensorFlow; оцените качество полученных моделей с помощью метрик; 
- реализуйте многослойный персептрон, с помощью которого можно решать задачи регрессии и классификации; предусмотрите возможность использовать такие функции активации, как sigmoid, tanh и relu; также предусмотрите возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой; реализуйте обучение персептрона методом обратного распространения ошибки; самостоятельно найдите производные функций sigmoid, tanh и relu; реализуйте классический градиентный спуск с возможностью указания шага.

# Задание 1

In [2]:
import sys
sys.path.insert(0, "C://Users//MSI//Desktop//OmSTU//MachineLearning//models//Neyron")
from multilayer_perceptron import MultilayerPerceptronModel

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [3]:
classification_data = pd.read_csv("../data/card_transdata.csv").sample(10000)
regression_data = pd.read_csv("../data/kc_house_data_preprocessed.csv").drop(columns=["Unnamed: 0"])

In [4]:
classification_target = classification_data['fraud']
classification_predicators = classification_data.drop(columns=['fraud'])
regression_target = regression_data['price']
regression_predicators = regression_data.drop(columns=['price'])

In [5]:
X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(
    classification_predicators, classification_target, test_size=0.5
)

X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(
    regression_predicators, regression_target, test_size=0.5
)

In [6]:
smote = SMOTE()
X_train_classification, y_train_classification = smote.fit_resample(X_train_classification, y_train_classification)
values = y_train_classification.value_counts()
print(values)
print(f"Отношение 0 к 1: { values[0] / values[1] }")

0.0    4589
1.0    4589
Name: fraud, dtype: int64
Отношение 0 к 1: 1.0


In [7]:
X_train_classification = StandardScaler().fit_transform(X_train_classification)
X_test_classification = StandardScaler().fit_transform(X_test_classification)

In [9]:
classification_model = tf.keras.Sequential(
    [
    tf.keras.layers.Dense(128, activation='relu', input_shape=(7,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)

In [10]:
classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")
classification_model.fit(X_train_classification, y_train_classification, epochs=20)

Epoch 1/20
287/287 [==============================] - 1s 2ms/step - loss: 0.0329
Epoch 2/20
287/287 [==============================] - 0s 2ms/step - loss: 0.0179
Epoch 3/20
287/287 [==============================] - 0s 1ms/step - loss: 0.0119
Epoch 4/20
287/287 [==============================] - 0s 1ms/step - loss: 0.0127
Epoch 5/20
287/287 [==============================] - 0s 1ms/step - loss: 0.0096
Epoch 6/20
287/287 [==============================] - 0s 2ms/step - loss: 0.0118
Epoch 7/20
287/287 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 8/20
287/287 [==============================] - 0s 1ms/step - loss: 0.0070
Epoch 9/20
287/287 [==============================] - 1s 2ms/step - loss: 0.0087
Epoch 10/20
287/287 [==============================] - 1s 2ms/step - loss: 0.0069
Epoch 11/20
287/287 [==============================] - 1s 2ms/step - loss: 0.0078
Epoch 12/20
287/287 [==============================] - 1s 2ms/step - loss: 0.0059
Epoch 13/20
287/287 [====

In [11]:
y_pred = np.around(classification_model.predict(X_test_classification))
print(classification_report(y_pred, y_test_classification))

157/157 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93      4020
         1.0       0.98      0.42      0.59       980

    accuracy                           0.88      5000
   macro avg       0.93      0.71      0.76      5000
weighted avg       0.90      0.88      0.86      5000



In [12]:
from joblib import dump
dump(classification_model, "neyron.joblib")

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-06-05 01:05:07         2929
metadata.json                                  2023-06-05 01:05:07           64
variables.h5                                   2023-06-05 01:05:07        96784


['neyron.joblib']

In [12]:
classification_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1024      
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 5,697
Trainable params: 5,697
Non-trainabl

In [13]:
regression_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(128, activation="relu", input_shape=(21,)),
        tf.keras.layers.Dense(32, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [14]:
regression_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")
regression_model.fit(X_train_regression, y_train_regression, epochs=50)

Epoch 1/50
338/338 [==============================] - 2s 2ms/step - loss: 102782566400.0000
Epoch 2/50
338/338 [==============================] - 1s 2ms/step - loss: 68260298752.0000
Epoch 3/50
338/338 [==============================] - 1s 2ms/step - loss: 64893775872.0000
Epoch 4/50
338/338 [==============================] - 1s 2ms/step - loss: 62885531648.0000
Epoch 5/50
338/338 [==============================] - 1s 2ms/step - loss: 64755675136.0000
Epoch 6/50
338/338 [==============================] - 1s 2ms/step - loss: 62003982336.0000
Epoch 7/50
338/338 [==============================] - 1s 2ms/step - loss: 61917638656.0000
Epoch 8/50
338/338 [==============================] - 1s 2ms/step - loss: 60020531200.0000
Epoch 9/50
338/338 [==============================] - 1s 2ms/step - loss: 62091304960.0000
Epoch 10/50
338/338 [==============================] - 1s 2ms/step - loss: 59034267648.0000
Epoch 11/50
338/338 [==============================] - 1s 2ms/step - loss: 59963043840.0

In [15]:
y_pred = np.around(regression_model.predict(X_test_regression))
print(mean_absolute_error(y_pred, y_test_regression))

338/338 [==============================] - 1s 2ms/step
163173.99574349957


In [16]:
regression_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               2816      
                                                                 
 dense_5 (Dense)             (None, 32)                4128      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 7,489
Trainable params: 7,489
Non-traina

# Задание 2

In [7]:
layers = ([32, "tanh"],
          [16, "tanh"],
          [8, "tanh"],
          [4, "tanh"],
          [1, "sigmoid"])

mlp = MultilayerPerceptronModel(layers,
                                l_rate=1e-8,
                                n_epoch=50)
mlp.fit(np.array(X_train_classification), np.array([y_train_classification]).T)
y_pred = mlp.predict(np.array(X_test_classification))
print(classification_report(y_pred, y_test_classification))

              precision    recall  f1-score   support

           0       0.60      0.99      0.74      2780
           1       0.91      0.17      0.28      2220

    accuracy                           0.62      5000
   macro avg       0.75      0.58      0.51      5000
weighted avg       0.73      0.62      0.54      5000



In [9]:
layers = ([32, "tanh"],
          [16, "linear"],
          [8, "tanh"],
          [4, "tanh"],
          [1, "linear"])

mlp = MultilayerPerceptronModel(layers, 
                                task="regression", 
                                l_rate=0.1, 
                                n_epoch=50, 
                                regularize='lasso', 
                                alpha=0.25)
mlp.fit(np.array(X_train_regression), np.array([y_train_regression]).T)
y_pred = mlp.predict(np.array(X_test_regression))
print(mean_absolute_error(y_pred, y_test_regression))

240775.4300336094
